In [2]:
from llama_cpp import Llama
import numpy as np
import json

In [3]:
import json
import time
import io
import ctypes
from contextlib import redirect_stderr
from llama_cpp import (
    Llama,
    llama_log_set,
)
from collections import OrderedDict
from typing import (
    Dict,
)


In [4]:

# suppressing + logging stdout/stderr -----

llama_log_obj = []

def suppress_stderr(func):
    def wrapper(*args, **kwargs):
        capture_stderr = io.StringIO()
        with redirect_stderr(capture_stderr):
            result = func(*args, **kwargs)
            llama_log_obj.append(capture_stderr.getvalue())
        return result
    return wrapper

def my_log_callback(level, message, user_data):
    llama_log_obj.append(message.decode())

log_callback = ctypes.CFUNCTYPE(None, ctypes.c_int, 
            ctypes.c_char_p, ctypes.c_void_p)(my_log_callback)

llama_log_set(log_callback, ctypes.c_void_p())


In [5]:
@suppress_stderr
def bar():
    return Llama(
        model_path="../data/mistral-7b-instruct-v0.2.Q4_K_M.gguf", 
        logits_all=True,
        
    )

In [6]:
llm = bar()

In [7]:
output = llm(
    "The capital of France is", 
    max_tokens=1, 
    temperature=2.0,
    logprobs=10,          # request top-k logprobs for tokens
    echo=False,
)

In [8]:
output

{'id': 'cmpl-0a327e18-50a0-4c33-89c2-02b715d56698',
 'object': 'text_completion',
 'created': 1755604940,
 'model': '../data/mistral-7b-instruct-v0.2.Q4_K_M.gguf',
 'choices': [{'text': ' not',
   'index': 0,
   'logprobs': {'tokens': [' not'],
    'text_offset': [24],
    'token_logprobs': [np.float32(-3.930922)],
    'top_logprobs': [{' Paris': np.float32(-1.416907),
      ' a': np.float32(-1.8824908),
      ' one': np.float32(-2.3024373),
      ' known': np.float32(-2.4046102),
      ' an': np.float32(-3.0555158),
      ' the': np.float32(-3.2259011),
      ' famous': np.float32(-3.4587197),
      ' home': np.float32(-3.5714765),
      ' also': np.float32(-3.7625794),
      ' not': np.float32(-3.930922)}]},
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 6, 'completion_tokens': 1, 'total_tokens': 7}}

In [9]:
top_logprobs = output['choices'][0]['logprobs']['top_logprobs'][0]

In [10]:
top_logprobs

{' Paris': np.float32(-1.416907),
 ' a': np.float32(-1.8824908),
 ' one': np.float32(-2.3024373),
 ' known': np.float32(-2.4046102),
 ' an': np.float32(-3.0555158),
 ' the': np.float32(-3.2259011),
 ' famous': np.float32(-3.4587197),
 ' home': np.float32(-3.5714765),
 ' also': np.float32(-3.7625794),
 ' not': np.float32(-3.930922)}

In [11]:
tmp = {k: np.exp(v) for k,v in top_logprobs.items()}
np.sum(np.array(list(tmp.values())))

np.float32(0.77424085)

In [12]:
@suppress_stderr
def foo(num_output=10):
    output = llm(
        "The capital of France is", 
        max_tokens=1, 
        # temperature=2.0,
        logprobs=num_output,          # request top-k logprobs for tokens
        echo=False
    )
    top_logprobs = output['choices'][0]['logprobs']['top_logprobs'][0]
    tmp = {k: np.exp(v) for k,v in top_logprobs.items()}
    return np.sum(np.array(list(tmp.values())))

In [13]:
foo(num_output=100)

np.float32(0.9547854)

In [ ]:
1+1

2

: 